# Description

**TODO**

# Modules loading

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [ ]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = True

In [ ]:
PREDICTION_METHOD = "Gene-based"

In [ ]:
LINCS_DATA_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs"
display(LINCS_DATA_DIR)
assert LINCS_DATA_DIR.exists()

In [ ]:
SPREDIXCAN_DATA_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "spredixcan" / "raw"
display(SPREDIXCAN_DATA_DIR)
assert SPREDIXCAN_DATA_DIR.exists()

In [ ]:
OUTPUT_PREDICTIONS_DIR = Path(LINCS_DATA_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

# Load PharmacotherapyDB gold standard

In [ ]:
gold_standard = pd.read_pickle(
    Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "gold_standard.pkl"),
)

In [ ]:
display(gold_standard.shape)

In [ ]:
display(gold_standard.head())

In [ ]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Raw data

In [ ]:
input_file = Path(LINCS_DATA_DIR, "lincs-data.pkl").resolve()

display(input_file)

In [ ]:
lincs_projection = pd.read_pickle(input_file)

In [ ]:
display(lincs_projection.shape)

In [ ]:
display(lincs_projection.head())

# Load S-PrediXcan

In [ ]:
phenomexcan_input_file_list = [
    f for f in SPREDIXCAN_DATA_DIR.glob("*.pkl") if f.name.startswith("spredixcan-")
]

In [ ]:
display(len(phenomexcan_input_file_list))

In [ ]:
pd.read_pickle(phenomexcan_input_file_list[0]).head()

# Predict drug-disease associations

In [ ]:
from drug_disease import predict_dotprod_neg

In [ ]:
methods_to_run = [predict_dotprod_neg]

In [ ]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)

    # get common genes with lincs
    common_genes = phenomexcan_projection.index.intersection(lincs_projection.index)
    phenomexcan_projection = phenomexcan_projection.loc[common_genes]
    lincs_projection = lincs_projection.loc[common_genes]

    print(f"  shape: {phenomexcan_projection.shape}")

    for prediction_method in methods_to_run:
        for ntc in (None, 50, 100, 250, 500):
            prediction_method(
                lincs_projection,
                phenomexcan_input_file,
                phenomexcan_projection,
                OUTPUT_PREDICTIONS_DIR,
                PREDICTION_METHOD,
                doids_in_gold_standard,
                FORCE_RUN,
                n_top_conditions=ntc,
                use_abs=True,
            )

            print("\n")